In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

2023-04-06 20:21:09.333970: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
data = pd.read_csv('balanced_credit.csv')

X = data.drop('default', axis=1).values
y = data['default'].values

# SPlit data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Scale features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Build model structure
model = Sequential([
    Dense(16, activation='relu'),
    Dropout(0.2),
    Dense(8, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])

# Compile and fit model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, validation_split=0.2, epochs=25, batch_size=32, verbose=1)

# make predictions and output metrics
y_pred = (model.predict(X_test) > 0.5).astype("int32")
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: {:.2f}%".format(accuracy * 100))

Epoch 1/50


2023-04-06 20:22:40.339984: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


266/266 [==============================] - 1s 2ms/step - loss: 0.6751 - accuracy: 0.5589 - val_loss: 0.6261 - val_accuracy: 0.6478
Epoch 2/25
266/266 [==============================] - 0s 2ms/step - loss: 0.6395 - accuracy: 0.6156 - val_loss: 0.6111 - val_accuracy: 0.6704
Epoch 3/25
266/266 [==============================] - 1s 2ms/step - loss: 0.6286 - accuracy: 0.6423 - val_loss: 0.6019 - val_accuracy: 0.6911
Epoch 4/25
266/266 [==============================] - 1s 2ms/step - loss: 0.6179 - accuracy: 0.6502 - val_loss: 0.5962 - val_accuracy: 0.6977
Epoch 5/25
266/266 [==============================] - 1s 2ms/step - loss: 0.6175 - accuracy: 0.6647 - val_loss: 0.5946 - val_accuracy: 0.7001
Epoch 6/25
266/266 [==============================] - 1s 2ms/step - loss: 0.6122 - accuracy: 0.6786 - val_loss: 0.5891 - val_accuracy: 0.6996
Epoch 7/25
266/266 [==============================] - 0s 2ms/step - loss: 0.6065 - accuracy: 0.6824 - val_loss: 0.5849 - val_accuracy: 0.7029
Epoch 8/25
266/26

In [3]:
data = pd.read_csv('balanced_credit.csv')

# One-hot encoding
data['EDUCATION'] = data['EDUCATION'].map(lambda x: 0 if x in [1, 2] else 1)
data['MARRIAGE'] = data['MARRIAGE'].map(lambda x: 0 if x == 1 else 1)
data['PAY_6'] = data['PAY_6'].map(lambda x: 0 if x in [-1, 1, 2] else 1)

X = data.drop('default', axis=1).values
y = data['default'].values

# Convert binary labels to categorical labels with two classes
y = np.array([[1, 0] if label == 0 else [0, 1] for label in y])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Build the neural network
model = Sequential([
    Dense(25, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(25, activation='relu'),
    Dense(2, activation='softmax')
])

model.compile(optimizer=Adam(learning_rate=0.0005), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, validation_split=0.2, epochs=50, batch_size=32, verbose=1)

y_pred = model.predict(X_test).argmax(axis=1)
y_test_labels = y_test.argmax(axis=1)
accuracy = accuracy_score(y_test_labels, y_pred)

print("Accuracy: {:.2f}%".format(accuracy * 100))

266/266 [==============================] - 8s 26ms/step - loss: 0.6362 - accuracy: 0.6357 - val_loss: 0.6055 - val_accuracy: 0.6780
Epoch 2/50
266/266 [==============================] - 6s 24ms/step - loss: 0.6025 - accuracy: 0.6822 - val_loss: 0.5931 - val_accuracy: 0.6907
Epoch 3/50
266/266 [==============================] - 7s 25ms/step - loss: 0.5919 - accuracy: 0.6919 - val_loss: 0.5849 - val_accuracy: 0.6982
Epoch 4/50
266/266 [==============================] - 7s 26ms/step - loss: 0.5848 - accuracy: 0.6974 - val_loss: 0.5829 - val_accuracy: 0.6944
Epoch 5/50
266/266 [==============================] - 6s 24ms/step - loss: 0.5803 - accuracy: 0.6963 - val_loss: 0.5797 - val_accuracy: 0.6977
Epoch 6/50
266/266 [==============================] - 6s 24ms/step - loss: 0.5763 - accuracy: 0.7013 - val_loss: 0.5812 - val_accuracy: 0.6911
Epoch 7/50
266/266 [==============================] - 6s 24ms/step - loss: 0.5735 - accuracy: 0.7005 - val_loss: 0.5779 - val_accuracy: 0.7006
Epoch 8/50

2023-04-06 20:28:15.630819: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


83/83 [==============================] - 0s 3ms/step
Accuracy: 70.70%


In [7]:
import itertools
from tensorflow.keras import backend
from scikeras.wrappers import KerasClassifier
import time

data = pd.read_csv('balanced_credit.csv')

X = data.drop('default', axis=1).values
y = data['default'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

def create_model(learning_rate=0.001, neurons=64, dropout_rate=0.2):
    backend.clear_session()
    optimizer = Adam(learning_rate=learning_rate)
    if dropout_rate is None:
        model = Sequential([
            Dense(neurons, activation='relu', input_shape=(X_train.shape[1],)),
            Dense(neurons // 2, activation='relu'),
            Dense(1, activation='sigmoid')
        ])
    else:
        if neurons < 128:
            model = Sequential([
                Dense(neurons, activation='relu', input_shape=(X_train.shape[1],)),
                Dropout(dropout_rate),
                Dense(neurons // 2, activation='relu'),
                Dropout(dropout_rate),
                Dense(1, activation='sigmoid')
            ])
        else:
            model = Sequential([
                Dense(neurons, activation='relu', input_shape=(X_train.shape[1],)),
                Dropout(dropout_rate * 2),
                Dense(neurons // 2, activation='relu'),
                Dropout(dropout_rate * 2),
                Dense(neurons // 4, activation='relu'),
                Dropout(dropout_rate),
                Dense(1, activation='sigmoid')
            ])
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    return model

# Hyperparameter grid
learning_rates = [0.001]
neurons = [16, 32, 64, 128]
dropout_rates = [None, 0.2]
epochs = [25, 50]

combinations = list(itertools.product(learning_rates, neurons, dropout_rates, epochs))

best_accuracy = 0
best_params = None

for combo in combinations:
    start_time = time.time()

    learning_rate, neuron, dropout_rate, epoch = combo
    model = KerasClassifier(model=create_model, epochs=epoch, batch_size=32, verbose=0,
                            learning_rate=learning_rate, neurons=neuron, dropout_rate=dropout_rate)

    model.fit(X_train, y_train)
    accuracy = model.score(X_test, y_test)
    print(f"Accuracy: {accuracy:.4f} with learning_rate={learning_rate}, neurons={neuron}, dropout_rate={dropout_rate}, epochs={epoch}")

    elapsed_time = (time.time() - start_time) / 60
    print(f"Elapsed time: {elapsed_time:.2f} minutes")

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_params = combo

print(f"Best accuracy: {best_accuracy:.4f} with learning_rate={best_params[0]}, neurons={best_params[1]}, dropout_rate={best_params[2]}, epochs={best_params[3]}")

Accuracy: 0.6960 with learning_rate=0.001, neurons=16, dropout_rate=None, epochs=25
Elapsed time: 0.17 minutes
Accuracy: 0.6960 with learning_rate=0.001, neurons=16, dropout_rate=None, epochs=50
Elapsed time: 0.37 minutes
Accuracy: 0.6991 with learning_rate=0.001, neurons=16, dropout_rate=0.2, epochs=25
Elapsed time: 0.17 minutes
Accuracy: 0.7024 with learning_rate=0.001, neurons=16, dropout_rate=0.2, epochs=50
Elapsed time: 0.30 minutes
Accuracy: 0.6874 with learning_rate=0.001, neurons=32, dropout_rate=None, epochs=25
Elapsed time: 0.20 minutes


KeyboardInterrupt: 